In [568]:
import pandas as pd

In [569]:
csv_1 = pd.read_csv('Contructure.csv', index_col=0)[:3498]

In [570]:
csv_1.rationales

0       1. Restate the Question: The question is askin...
1       2. Restate the Question: The question is askin...
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
3493    4. Premise: The yellow dog is running on the d...
3494    5. Premise: Two women talk and laugh while peo...
3495    6. Premise: Two little boys swimming in the po...
3496    7. Premise: A man with gray hair and wearing g...
3497    8. Premise: People use the computer terminals ...
Name: rationales, Length: 3498, dtype: object

In [571]:
csv_2 = pd.read_csv('Contructure2.csv', index_col=0, sep='|')
csv_2

,premise,hypothesis,rationales
3498,"A party tent, balloons, and people are in the ...",The people are outside at a party.,"1. Premise: A party tent, balloons, and people..."
3499,A man dressed as a woman sitting behind a sewi...,A man with feminine attire sits with a sewing ...,NaN
3500,A partially balding man pushes a blue easy-cha...,A man is pushing a chair home.,NaN
3501,Two men holding skateboards and one man skateb...,A group of friends play a board game in the li...,NaN
3502,Two Asian women are embracing next to a car.,Two women are standing next to a street.,NaN
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,10. Restate the Question: What is the most lik...
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,NaN
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,NaN
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,NaN


In [572]:
csv_1 = pd.concat([csv_1, csv_2])
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Restate the Question: The question is askin...
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Restate the Question: The question is askin...
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,NaN
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,NaN
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,NaN
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,10. Restate the Question: What is the most lik...
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,NaN
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,NaN
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,NaN


In [573]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [574]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
What is the most likely answer to the premise '%s' and hypothesis %s with options "entailment", "neutral", "contradiction"?. \n
- Restate the Question: Begin by summarizing the question in your own words to ensure you've captured its essence.
- List the Choices: Enumerate the options provided for answering the question.
- Eliminate Irrelevant Choices: Identify and discard choices that are clearly not applicable, explaining why each is eliminated.
- Analyze Remaining Choices: Discuss the merits and drawbacks of the remaining choices in the context of the question.
- Select the Best Answer: Choose the most appropriate answer based on your analysis.
- Justify Your Choice: Provide a rationale for why you believe your chosen answer is the most fitting.
- Summarize: Conclude with a summary that encapsulates your rationale.
'''

In [581]:
error = csv_1[csv_1.rationales.isna() | csv_1.rationales.str.len().lt(10)]

In [582]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

<ipython-input-582-40c85959ab3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)


In [584]:
error.to_csv('error.csv')

In [578]:
print(error.prompt[:10].values)

['\nPremise: Dog leaps from water while woman kneels on the dock playing with him, \nHypothesis: A woman and dog sit on a dock together..\nWhat is the most likely answer to the premise \'Dog leaps from water while woman kneels on the dock playing with him\' and hypothesis A woman and dog sit on a dock together. with options "entailment", "neutral", "contradiction"?. \n\n- Restate the Question: Begin by summarizing the question in your own words to ensure you\'ve captured its essence.\n- List the Choices: Enumerate the options provided for answering the question.\n- Eliminate Irrelevant Choices: Identify and discard choices that are clearly not applicable, explaining why each is eliminated.\n- Analyze Remaining Choices: Discuss the merits and drawbacks of the remaining choices in the context of the question.\n- Select the Best Answer: Choose the most appropriate answer based on your analysis.\n- Justify Your Choice: Provide a rationale for why you believe your chosen answer is the most 

In [579]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

AttributeError: 'float' object has no attribute 'lower'

In [463]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [464]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [467]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory', 'entail', 'contradict']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [468]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [469]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,split,correct_index,LLM_answer


In [481]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [470]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [471]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [472]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [473]:
paper.shape

(5592, 6)

In [474]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,"{'contradiction': 193, 'contradictory': 411}",contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'contradiction': 120},contradiction
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'entailment': 143},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entailment': 179},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'neutral': 119},neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...,8. premise: three women walk down a city stree...,"{'contradiction': 143, 'contradicts': 323}",contradiction
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...,9. premise: a basketball player slam dunks the...,{'entailment': 194},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...,10. premise: two kids wearing white robes are ...,{'entailment': 235},entailment


In [475]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'entail', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradict', 'LLM_answer'] = 'contradiction'

In [476]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,"{'contradiction': 193, 'contradictory': 411}",contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'contradiction': 120},contradiction
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'entailment': 143},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entailment': 179},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'neutral': 119},neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...,8. premise: three women walk down a city stree...,"{'contradiction': 143, 'contradicts': 323}",contradiction
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...,9. premise: a basketball player slam dunks the...,{'entailment': 194},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...,10. premise: two kids wearing white robes are ...,{'entailment': 235},entailment


In [477]:
paper['my_label'] = total_df['LLM_answer']

In [478]:
sum(paper.label == paper.my_label) / len(paper)

0.6194563662374821

In [480]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [107]:
total_df.to_csv('historical - full.csv')

In [195]:
csv_1.to_csv('neutral - full.csv', index=False)

In [162]:
total_df.index.value_counts()

A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.                                                                     1
a man taking a pizza out of an oven</s>A woman is sitting in the oven.                                                                                                                                                                  1
Side view of a mountaineer walking into strong headwinds.</s>A man outside.                                                                                                                                                             1
A boy jumping off of a diving board while a girl watches.</s>A boy is near water.                                                                                                                                                       1
A back view of a man driving a tractor and signaling with two ch